<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/BatchData_SkipTracing_MockData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch Data Skip Tracing Mock Data

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 1st Jun 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions
  - Signup for BatchData
  - Create mock api key and live api key

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [BatchData API docs](https://developer.batchdata.com/docs/batchdata/a45e094c668b1-property-skip-trace)
- [Curl Convertor](https://curlconverter.com/#)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [ ]:

import pandas as pd
import requests
import os
import warnings

pd.set_option('display.max_columns', None) # show all columns

## <font color="blue">Functions</font>

## <font color="blue">Locals & Constants</font>

In [ ]:
drive.mount('/content/drive')

In [ ]:
# set working directory
os.chdir('/content/drive/MyDrive/Colab Data/')
dir = os.getcwd()
print('Current working directory:', dir)

# remove pandas warnings
warnings.filterwarnings('ignore')

## <font color="blue">Data</font>

In [ ]:
# read in api keys
df_api_keys = pd.read_csv('batchapi.csv')
# extract api key
batchdata_mock_api_key = df_api_keys.loc[df_api_keys['API'] == 'batchdata_mock']['KEY'].iloc[0]

## <font color="blue">Transformations</font>

### <font color="green">1) Single Property - Skip Trace Search</font>

In [ ]:
bt = "3c8ZEz3xSqLIJidUzAmkfTjtwGCfFIfraDsckGPM"

In [5]:
url = 'https://stoplight.io/mocks/batchdata/batchdata/20349728/property/skip-trace'

json_data =  {
  "requests": [
    {
      "propertyAddress": {
        "city": "Franklin Square",
        "street": "1011 Rosegold St",
        "state": "NY",
        "zip": "11010-2507"
      }
    },
    {
      "propertyAddress": {
        "street": "25866 W Globe Ave",
        "city": "Buckeye",
        "state": "AZ",
        "zip": "85326"
      }
    }
  ]
}


headers = {
  'Content-Type': "application/json",
  'Authorization': 'Bearer ' + bt
}

response = requests.post(url, headers=headers, json=json_data)
response.text

'{"status":{"code":200,"text":"OK"},"results":{"persons":[{"_id":"tOdljN72SozLw","bankruptcy":{},"dnc":{},"emails":[{"email":"johndoe@gmail.net"}],"mailingAddress":{},"name":{"first":"john","last":"doe"},"phoneNumbers":[{"number":"111111111","carrier":"OMNIPOINT COMMUNICATIONS- INC.","type":"Mobile","tested":true,"reachable":true,"score":100},{"number":"2222222222","carrier":"VERIZON NEW YORK- INC.","type":"Land Line","tested":false,"reachable":false,"score":95},{"number":"333333333","carrier":"VERIZON NEW YORK- INC.","type":"Land Line","tested":true,"reachable":true,"score":90},{"number":"444444444","carrier":"VERIZON NEW YORK- INC.","type":"Land Line","tested":true,"reachable":true,"score":85}],"litigator":false,"propertyAddress":{"houseNumber":"1011","street":"1011 Rosegold St","city":"Franklin Square","county":"Nassau","state":"NY","zip":"11010","zipPlus4":"2507","formattedStreet":"Rosegold St","streetNoUnit":"1011 Rosegold St","hash":"b7bd4cea51b26af459febfbf822c99e2"},"involuntar

In [ ]:
response.json()['results']

In [ ]:
_df = pd.json_normalize(response.json()['results']['persons'])
print('No of columns:', len(_df.columns))
_df

In [ ]:
df_phone_num = pd.DataFrame(_df["phoneNumbers"].iloc[0])
df_phone_num

In [ ]:
# filter on valid phone numbers
df_ph_valid = df_phone_num.loc[
  (df_phone_num['reachable'] == True) & (df_phone_num['score'] >= 90)]
# change phone numbers for testing purposes only
df_ph_valid['number'] = [6027828692, 5633574823] # mock dnc phone numbers
df_ph_valid['number'] = df_ph_valid['number'].astype(str)
df_ph_valid

In [ ]:
url = "https://stoplight.io/mocks/batchdata/batchdata/20349728/phone/dnc"

json_data = {
  'requests': df_ph_valid['number'].tolist(), # pass all valid numbers
}

headers = {
  'Content-Type': "application/json",
  'Authorization': 'Bearer ' + batchdata_mock_api_key
}

dnc_response = requests.post(url, headers=headers, json=json_data)
dnc_response.json()

In [ ]:
# transform JSON object to a DataFrame and select relevant columns
df_dnc = pd.json_normalize(dnc_response.json()['results']['phoneNumbers'])[['number', 'dnc']]
df_dnc

In [ ]:
# convert numbers to string type
df_ph_valid['number'] = df_ph_valid['number'].astype(str)
df_dnc['number'] = df_dnc['number'].astype(str)
# merge valid and dnc tables
df_ph_valid_dnc = pd.merge(df_ph_valid, df_dnc, how='left', on=['number'])
df_ph_valid_dnc

In [ ]:
# filter on phone numbers that are NOT on DNC (Do Not Call) list
df_ph_valid_dnc_fltr = df_ph_valid_dnc.loc[df_ph_valid_dnc['dnc'] == False]
df_ph_valid_dnc_fltr

In [ ]:
# create copy of df
df = _df.copy()

# create phone numbers columns
i = 1
# itereate through list of valid numbers
for num in df_ph_valid_dnc_fltr['number'].tolist():
  df['phone' + str(i)] = [num] # create column

# create email column
df['emails'] = df.apply(lambda x: x['emails'][0], axis=1)
df = pd.concat([df, df["emails"].apply(pd.Series)], axis=1)
df

In [ ]:
output_cols = ['name.first', 'name.last',
       'propertyAddress.street', 'propertyAddress.city',
       'propertyAddress.county', 'propertyAddress.state',
       'propertyAddress.zip', 'property.equity',
       'property.equityPercent', 'property.absenteeOwner', 'property.vacant',
       'property.uspsDeliverable', 'phone1', 'email']

df_output = df[output_cols]
df_output

### <font color="green">2) Multi Property - Skip Trace Search</font>

## <font color="blue">Output</font>

In [ ]:
# download file
df_output.to_csv('output.csv', index=False)
files.download('output.csv')

# End Notebook